# P1
Written by Babak Hosseini Parsa Ahmadi 810101609, Mohtasham 810101408 and Mahdy Naeini 810101536\
Data science project abou digikala mobile phones.

In [1]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import random
import math
import scipy.stats as stats
import numpy  as np
import seaborn as sns
import datetime
import time
import re

# Scraping DigiKala

In [2]:
SUB_FEATURES = {"مدل": "Model", "زمان معرفی": "Release Data","وزن": "Weight", "ابعاد": "Dimensions", "برند": "Brand","فروشنده":"Seller",
                "توضیحات سیم کارت":"Sim Card Description", "نوع سیم کارت":"Sim Card Type", "ساختار بدنه":"Body Structure",
                "ویژگی‌های کلیدی": "Key Features", "پردازنده‌ مرکزی": "CPU", "فرکانس پردازنده‌ مرکزی":"CPU Clock Rate",
                "پردازنده‌ گرافیکی":"GPU","توضیحات کارت حافظه جانبی":"SD Card Description","بازه‌ اندازه صفحه نمایش":"Range of Screen Size",
                "نسبت تصویر":"DAR","سایر قابلیت‌ها":"Other Screen Features","موقعیت نما ( GPS )":"GPS","نوع Wi-Fi (ورژن)":"Wi-Fi Type",
                "بلوتوث":"Bluetooth","نسخه بلوتوث":"Bluetooth Version","درگاه‌های ارتباطی":"Ports","رزولوشن عکس":"Camera Resolution",
                "فناوری فوکوس":"Focus Technology","فلش":"Flashlight","قابلیت‌های دوربین":"Camera Features","فیلمبرداری":"Video Recording",
                "خروجی صدا":"Audio Output","سیستم عامل":"OS","نسخه سیستم عامل":"OS Version","حس‌گرها":"Sensors",
                "مشخصات باتری": "Battery Spec","اقلام همراه":"Accessories","سایر توضیحات اقلام همراه":"Accessories Description",
                "دسته ‌بندی": "Category", "تراشه":"Chipset","حافظه داخلی":"Internal Memory","مقدار RAM":"RAM","گارانتی":"Warranty",
                "پشتیبانی از کارت حافظه": "SD Card Support", "فناوری صفحه‌ نمایش": "Display Technology", "قیمت نهایی": "Price Final",
                "اندازه":"Screen Size","رزولوشن صفحه نمایش":"Screen Resolution","تراکم پیکسلی":"Pixel Density","توضیحات تکمیلی صدا":"Supplementary Audio Description",
                "نسبت صفحه‌ نمایش به بدنه": "Ratio of Screen Size to Body", "شبکه‌های ارتباطی": "Cellular Network",
                "شبکه‌های مخابراتی و ارتباطی قابل پشتیبانی":"Supported Networks","دوربین‌های پشت گوشی":"Back Facing Cameras",
                "دوربین سلفی": "Front Facing Cameras", "رنگ های موجود": "Available Colors", "قیمت بدون تخفیف": "Price No Discount",
                "ویژگی‌های خاص": "Special Features","تعداد سیم کارت":"Number of Sim Cards","امتیاز خریداران":"Score"}

In [3]:
def make_new_csv(file_addr):
    with open(file_addr, mode='w') as mobiles_file:
        mobilesCSV = csv.writer(mobiles_file)
        mobilesCSV.writerow(list(SUB_FEATURES.values()))

In [55]:
def write_to_csv(addr, driver,writer):
    driver.get("https://www.digikala.com/"+addr)
    sleep(1)
    while(True):
        try:
            spec_button=None
            spec_more_button=None
            while True:
                try:
                    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
                    sleep(1)
                    driver.execute_script('window.scrollTo(0, -document.body.scrollHeight);')
                    spec_button = driver.find_element(By.ID, "specification").find_elements(
                        By.XPATH, "./..")
                    sleep(1)
                    driver.execute_script("arguments[0].scrollIntoView();", spec_button[0])
                    sleep(1)
                    spec_more_button=spec_button[0].find_elements(By.TAG_NAME, "span")
                    spec_more_button[-2].click()
                    sleep(1)
                    break
                except (NoSuchElementException, ElementClickInterceptedException):
                    continue
            page = driver.page_source
            soup = bs(page, 'html.parser')

            global SUB_FEATURES
            scraped_sub_features = dict()
            features_div=soup.find(string="مشخصات کلی").parent.parent.parent
            for fa_feature, eng_feature in (SUB_FEATURES.items()):
                if eng_feature == "Accessories":
                    pass
                if features_div.find(string=fa_feature):
                    found=features_div.find(
                        string=fa_feature).parent.parent.findAll('p')[1:]
                    scraped_sub_features[eng_feature] = "\r\n".join([s.text for s in found])
                else:
                    scraped_sub_features[eng_feature] = ""

            colors_element=soup.find("div", {"class": "flex lg:flex-wrap overflow-x-auto px-5 lg:px-0"}
                    ).findAll("div", {"class": "whitespace-nowrap"})
            colors=[]
            for color in colors_element:
                colors.append(color.text)
            scraped_sub_features['Available Colors'] = colors

            score = soup.findAll("p", {"class": "mr-1 text-body-2"})
            if score:
                scraped_sub_features["Score"] = score[0].text
            
            brand = soup.findAll("a", {"data-cro-id": "pdp-breadcrumb-down"})
            if brand:
                scraped_sub_features["Brand"] = brand[0].text

            sellers=(soup.find("div", {"id": "sellerSection"}))
            if not sellers:
                price_no_discount = soup.findAll("span", {"data-testid": "price-no-discount"})
                for element in price_no_discount:
                    if element and " ".join(element.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                        scraped_sub_features["Price No Discount"] = element.text
                        break

                price_final = soup.findAll("span", {"data-testid": "price-final"})
                for element in price_final:
                    if element and " ".join(element.parent.parent.parent.parent.attrs["class"][0:2]) == "flex items-center":
                        scraped_sub_features["Price Final"] = element.text
                        break

                Warranty = soup.findAll(string=re.compile("گارانتی"))
                if Warranty:
                    scraped_sub_features["Warranty"] = Warranty[0].parent.text
                    
                scraped_sub_features["Seller"] = soup.find('h3', string=re.compile("فروشنده")).parent.parent.findChildren(
                    "div", recursive=False)[2].findChildren("p", recursive=True)[0]
                writer.writerow(list(scraped_sub_features.values()))
                break

            sellers_button = driver.find_element(By.ID, "sellerSection")
            driver.execute_script(
                "arguments[0].scrollIntoView();", sellers_button)
            sleep(1)
            sellers_more_button = sellers_button.find_elements(
                By.TAG_NAME, "span")
            sellers_more_button[-1].click()

            page = driver.page_source
            soup = bs(page, 'html.parser')
            sellers = (soup.find("div", {"id": "sellerSection"}))
            for seller in sellers.findChildren("div",recursive=False)[0].\
                findChildren("div",recursive=False)[1:]:
                seller_div=seller.findChildren("div", recursive=False)[0].findChildren("div", recursive=False)
                name=seller_div[0].findChildren("div", recursive=False)[1]
                scraped_sub_features["Seller"] = name.findChildren(
                    "div", recursive=False)[1].findChildren("div", recursive=False)[0].text
                warranty=seller_div[0].findChildren("div", recursive=False)[3]
                scraped_sub_features["Warranty"] = warranty.text
                price = seller_div[1].findChildren(
                    "div", recursive=False)[0].findChildren(
                    "div", recursive=False)[0].findChildren(
                    "div", recursive=False)[0]
                prices=price.findChildren(
                    "span", recursive=True)
                scraped_sub_features["Price No Discount"] = prices[-1].text
                if len(prices)==3:
                    scraped_sub_features["Price No Discount"] = prices[0].text
                    scraped_sub_features["Price Final"] = prices[1].text
                writer.writerow(list(scraped_sub_features.values()))
            break
        except (NoSuchElementException, ElementClickInterceptedException):
            driver.refresh()
            continue  

In [43]:
def scrape_list_of_mobiles(number_of_mobiles):
    driver = webdriver.Chrome()
    mobiles = []
    i = 1
    while len(mobiles) < number_of_mobiles:
        driver.get(
            f'https://www.digikala.com/search/category-mobile-phone/product-list/?has_selling_stock=1&page={i}')
        wait = WebDriverWait(driver, 60)
        wait.until(EC.presence_of_element_located((
            By.XPATH, "//div[@data-testid='product-card']")))
        page = driver.page_source
        soup = bs(page, 'html.parser')
        mobiles.extend(soup.find_all(
            'div', {'class': 'product-list_ProductList__item__LiiNI'}))
        driver.execute_script(
            'window.scrollTo(0, document.body.scrollHeight);')
        sleep(1)
        i += 1
    driver.close()
    return mobiles

In [44]:
def scrape_all_mobiles(file_addr,mobiles,start):
    driver = webdriver.Chrome()
    csv_file = open(file_addr, mode='a', encoding='utf-8')
    csv_writer=csv.writer(csv_file)
    for i in range(start,len(mobiles)):
        write_to_csv(mobiles[i].find('a')['href'], driver, csv_writer)
        i+=1
    driver.close()
    csv_file.close()

In [7]:
FILE_ADDR='digikala_phones.csv'

In [8]:
make_new_csv(FILE_ADDR)

In [11]:
mobiles = scrape_list_of_mobiles(278)

In [57]:
scrape_all_mobiles(FILE_ADDR,mobiles,7)

IndexError: list index out of range

# Preprocessing DigiKala

In [411]:
PREPROCESSING_MAP = {"دسامبر": 12, "نوامبر": 11, "اکتبر": 10, "سپتامبر": 9, "آگوست": 8, "جولای": 7, "ژوئن": 6, "می": 5, "آوریل": 4, "مارچ": 3, "مارس": 3,
                      "فوریه": 2, "ژانویه": 1,"دو":"2","سه":3,
                      "یک":"1",".":".","۰":"0","۱":"1","۲":"2","۳":"3","۴":"4","۵":"5","۶":"6","۷":"7","۸":"8","۹":"9"}

In [412]:
original_df=pd.read_csv(FILE_ADDR)
df=original_df.copy()

In [413]:
df.drop_duplicates(inplace=True)
target_cols = df.columns
multi_cols = list(filter(lambda x: df[x].astype(str).str.contains("\r\n").any(), df.columns))
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: "".join(
        [PREPROCESSING_MAP[c] if c in PREPROCESSING_MAP else c for c in x]).replace("\u200d", " ") if x is not np.nan and type(x) is not float else x).astype(object)
    if col in multi_cols:
        df.loc[:, col] = df[col].apply(lambda x: x.replace("\u200c", " ").split(
            "\r\n") if x is not np.nan and type(x) is not float else x)
df[target_cols]

,Model,Release Data,Weight,Dimensions,Brand,Sim Card Description,Sim Card Type,Body Structure,Key Features,CPU,...,Ratio of Screen Size to Body,Cellular Network,Supported Networks,Back Facing Cameras,Front Facing Cameras,Available Colors,Price No Discount,Special Features,Number of Sim Cards,Score
0,[iPhone 13 CH ],14 سپتامبر 2021,174 گرم,146.7x71.5x7.65 میلی‌متر,اپل,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب جلویی و پشتی از جنس شیشه, فریم از جنس آلو...",NaN,2x GHz Avalanche + 4x GHz Blizzard,...,86,"[2G , 3G , 4G , 5G ]","[GPRS , NFC , QZSS , Wi-Fi , بلوتوث ]",2 ماژول دوربین,"[دارای یک حسگر سلفی | دوربین 12 مگاپیکسلی, دور...","['مشکی', 'سفید', 'آبی', 'صورتی', 'سبز تیره']","37,650,000","[دارای بدنه مقاوم , مجهز به حس گر تشخیص چهره ,...",دو عدد,4.5
1,[Galaxy A15 ],11 دسامبر 2023,200 گرم,8.4×76.8×160.1 میلی‌متر,سامسونگ,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب جلو از جنس شیشه / , قاب پشت و فریم از جنس...","[صفحه نمایش با کیفیت / , دوربین اصلی مناسب / ,...",2x Cortex-A76 + 6x Cortex-A55,...,84.3,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",3 ماژول دوربین,[دارای حسگری با رزولوشن 13 مگاپیکسل از نوع عری...,"['زرد', 'سرمه ای', 'آبی روشن']","5,855,000","[مجهز به حس گر اثرانگشت , مناسب عکاسی ]",دو عدد,4.5
2,[Galaxy A05s ],25 سپتامبر 2023,194 گرم,8.8×77.8×168 میلی‌متر,سامسونگ,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,"[قاب جلو از جنس شیشه / , قاب پشت و فریم از جنس...",NaN,4x Kryo 265 Gold + 4x Kryo 265 Silver,...,82.9,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",3 ماژول دوربین,[دارای یک حسگر دوربین | دوربین با رزولوشن 13 ...,"['مشکی', 'نقره ای']","4,870,000","[مجهز به حس گر اثرانگشت , مناسب عکاسی ]",یک عدد,4.4
3,[105 2023 ],18 می 2023,78.7 گرم,14.5×49.4×115.1 میلی‌متر,نوکیا,[سایز اصلی (15 × 25 میلی متر) ],NaN,NaN,[باتری قدرتمند ],NaN,...,NaN,[2G ],[رادیو ],بدون دوربین,[ندارد ],['ذغالی'],"1,210,000",[کلاسیک ],دو عدد,4.3
4,[Redmi 13C ],10 نوامبر 2023,192 گرم,8.1×78×168 میلی‌متر,شیائومی,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,NaN,[رابط کاربری MIUI 14 ],2x Cortex-A75 + 6x Cortex-A55,...,83.7,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",2 ماژول دوربین,[دارای یک حسگر دوربین | دوربین با رزولوشن 8 م...,"['مشکی', 'سفید', 'آبی', 'سبز']","7,090,000","[مجهز به حس گر اثرانگشت , مناسب عکاسی ]",دو عدد,4.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,[nova 11i ],1 می 2023,193 گرم,8.55×75.55×164.6 میلی‌متر,هوآوی,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,[مقاوم در برابر پاشش قطرات آب ],[صفحه نمایش با نرخ بروزرسانی 90 هرتز ],4x Cortex-A73 + 4x Cortex-A53,...,90,"[2G , 3G , 4G ]","[QZSS , Wi-Fi , بلوتوث ]",2 ماژول دوربین,"[ - دارای یک حسگر دوربین / , - حسگر با رزولوش...","['مشکی', 'سبز']","8,399,000","[مجهز به حس گر اثرانگشت , مناسب عکاسی , مناسب ...",دو عدد,4.4
279,[Q20s ],NaN,208 گرم,9×75.9×164.54 میلی‌متر,جی پلاس,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,[قاب جلو و پشت از جنس شیشه فریم از جنس پلاستیک ],"[صفحه نمایش با نرخ بروزرسانی 60 هرتز / , طراح...",4xCortex-A53 + 4xCortex-A53,...,NaN,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",3 ماژول دوربین,[یک حسگر با رزولوشن 8 مگاپیکسل ],['مشکی'],"4,899,000",[مجهز به حس گر اثرانگشت ],دو عدد,NaN
280,[X20 ],NaN,204 گرم,8.9×76.3×164.75 میلی‌متر,جی پلاس,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,[قاب جلو و پشت از جنس شیشه فریم از جنس پلاستیک ],"[سنسور عریض مناسب , صفحه نمایش با نرخ بروزرسان...",4xCortex-A53 + 4xCortex-A53,...,NaN,"[2G , 3G , 4G ]","[Wi-Fi , بلوتوث ]",3 ماژول دوربین,[یک حسگر با رزولوشن 8 مگاپیکسل ],"['مشکی', 'آبی']","4,259,000","[مجهز به حس گر اثرانگشت , مناسب عکاسی ]",دو عدد,NaN
281,[Xiaomi Redmi Note 12 Turbo ],28 مارس 2023,181 گرم,161.1X75X7.9 میلی‌متر,شیائومی,[سایز نانو (8.8 × 12.3 میلی متر) ],NaN,[بخش جلو از شیشه / فریم پلاستیکی / بخش پشتی از...,[برای استفاده از گوشی به صورت گلوبال و پشتیبان...,Octa-core (1x2.91 GHz Cortex-X2 &amp; 3x2.49 G...,...,88.9,"[2G , 3G , 4G , 5G ]","[NFC , OTG ]",3 ماژول دوربین,[دارای 1 حسگر دوربین | دوربین با رزولوشن 16 مگ...,"['مشکی', 'سفید', 'آبی']","18,499,000","[دارای بدنه مقاوم , مجهز به حس گر اثرانگشت , م...",دو عدد,NaN


In [414]:
target_cols = ['Price No Discount', 'Price Final']
df['Price Final'].fillna(df['Price No Discount'], inplace=True)
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: int(
        x.replace(',', '') if x is not np.nan else x))
df[target_cols]

,Price No Discount,Price Final
0,37650000,37650000
1,5855000,5855000
2,4870000,4870000
3,1210000,1210000
4,7090000,7090000
...,...,...
278,8399000,8399000
279,4899000,4899000
280,4259000,4259000
281,18499000,18499000


In [415]:
target_cols = ['Weight', 'Bluetooth Version', 'Camera Resolution']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: (x.replace('%', '').replace(',',' ').split()[0]) if x is not np.nan else x)
df[target_cols]

,Weight,Bluetooth Version,Camera Resolution
0,174,5.0,12
1,200,5.3,50
2,194,5.1,50
3,78.7,NaN,NaN
4,192,5.3,50
...,...,...,...
278,193,5.0,48
279,208,5.0,13
280,204,5.0,50
281,181,5.3,64


In [416]:
target_cols = ['X Dimension', 'Y Dimension', 'Z Dimension']
for i,col in enumerate(target_cols):
    df.loc[:, col] = df['Dimensions'].apply(lambda x: float("".join(x.split()[0:-1]).replace('*', '×').replace('X', '×').replace('x', '×').split('×')[i])
                                            if x is not np.nan else x)
df[target_cols]

In [417]:
target_cols = ['Number of Sim Cards']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: (
        PREPROCESSING_MAP[x.split()[0]]) if x is not np.nan else x)
df[target_cols]

,Number of Sim Cards
0,2
1,2
2,1
3,2
4,2
...,...
278,2
279,2
280,2
281,2


In [418]:
target_cols = ['DAR']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: round(float(
        x.replace(' ','').split(':')[0])/float(
        x.replace(' ', '').split(':')[1]), 3) if x is not np.nan else x)
df[target_cols]

In [419]:
target_cols = ['X Screen Resolution',
               ' Y Screen Resolution']
for i, col in enumerate(target_cols):
    df.loc[:, col] = df['Screen Resolution'].apply(lambda x: 
                                                   (re.sub("[^0-9]"," ",x).strip(' ').split(' ')[-i])
                                                   if x is not np.nan else x)
df[target_cols]

,X Screen Resolution,Y Screen Resolution
0,1170,2532
1,1080,2340
2,1080,2400
3,120,160
4,720,1600
...,...,...
278,1080,2388
279,720,1600
280,720,1600
281,1080,2400


In [420]:
target_cols = ['Release Data']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: x.split()[2]+
                                   f", {PREPROCESSING_MAP[x.split()[1]]}" +
                                   f", {x.split()[0]}" if x is not np.nan else x)
df[target_cols]

In [421]:
target_cols = ['Release Data UNIX']
for col in target_cols:
    df.loc[:, col] = df['Release Data'].apply(
        lambda x: time.mktime(datetime.date(int(x.split(', ')[0]), int(x.split(', ')[1]), int(x.split(', ')[2])).timetuple()) 
        if x is not np.nan else x)
df[target_cols]

In [422]:
target_cols = ['Warranty']
for col in target_cols:
    df.loc[:, col] = df[col].apply(lambda x: x.split("ماهه")[1].split('(')[0].strip() if x is not np.nan else x)
df[target_cols]

In [ ]:
numerical_cols = df.select_dtypes(include=[np.number])
plt.figure(figsize=(10, 10))
sns.heatmap(numerical_cols.corr(), annot=True, cmap="Blues")
plt.title("Correlation Heatmap")
plt.show()

# Scraping Torob

In [ ]:
SUB_FEATURES = {"نام": "Title", "حافظه داخلی": "Internal Memory", "حافظه RAM": "RAM Memory", "کشور ROM": "ROM Country", "سیستم عامل": "OS",
                "سیم کارت": "Sim Card", "وزن": "Weight", "فناوری صفحه‌نمایش": "Display Technology", "دوربین‌های پشت گوشی": "Back Facing Cameras",
                "بازه‌ اندازه صفحه": "Range of Screen Size", "توضیحات سیم کارت": "Sim Card Description", "اندازه": "Size",
                "رزولوشن": "Resolution", "تعداد سیم کارت": "Number of Sim Cards", "سایر قابلیت‌ها": "Other Features", "ظرفیت باتری":"Battery Capacity",
                "کیفیت دوربین اصلی":"Main Camera Resolution",
                "رزولوشن عکس": "Camera Resolution", "میزان شارژ آماده به کار": "Charge to Work", "میزان شارژ مکالمه": "Charge to Call",
                "ابعاد": "Dimensions", "استان": "District", "فروشگاه": "Shop", "قیمت": "Price", "فروشگاه آنلاین":"Is Online Shop"}